In [1]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import optimizers
import keras
from IPython.display import display as display_dataframe
# from keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.applications.densenet import DenseNet121
import cv2
import os
import seaborn as sns
# from keras.preprocessing.image import load_img, img_to_array


import warnings
warnings.filterwarnings('ignore')

def display_all_in_a_list(image_list, titles=None, scale_factor=1):
    '''
    image_list: a single image, or a list of images
    -----------------------------------
    uses matplotlib to display all images in 'image_list' in the same cell
    '''
    plt.rcParams['figure.figsize'] = [10*scale_factor, len(image_list)*5*scale_factor]
    _, axes = plt.subplots(len(image_list), 1)
    if len(image_list) == 1:
        axes.imshow(image_list[0])
        if titles is not None:
            axes.set_title(titles[0])
    else:
        for index in range(len(image_list)):
            axes[index].imshow(image_list[index])
            if titles is not None:
                axes[index].set_title(titles[index])
    plt.show()
    
def get_single_image(something, margin = 1):
    #check for correct input shape
    if len(something.shape) > 3:
        raise Exception("Input should be of format (x, y, channels). Got {}".format(something.shape))
    
    #calculate the output shape of our returned image
    channels = something.shape[2]
    rows = int(channels**0.5)
    columns = int(channels / rows)
    while channels % rows != 0:
        rows -= 1
    while channels % columns != 0:
        columns += 1
    print("{} rows and {} columns of shape {}.".format(rows, columns, something.shape[:2]))
    
    margin = margin
    value = 255
        
    # stack the columns
    listy = [something[:, :, i*columns] for i in range(rows)]
    for i in range(rows):
        for j in range(1, columns):
            if margin:
                listy[i] = np.append(listy[i], np.array([[value]*margin]*listy[i].shape[0]), axis=1)
            listy[i] = np.append(listy[i], something[:, :, i*columns + j], axis=1)
    
    #stack the rows
    image = listy[0]
    for i in range(1, rows):
        if margin:
            image = np.append(image, np.array([[value]*listy[i].shape[1]]*margin), axis=0)
        image = np.append(image, listy[i], axis=0)
    
    return image

Using TensorFlow backend.


# load data

In [2]:
#vgg can't take input of 28x28 so we shall resize all our images
pickle_folder = "pickle files"
input_shape=(32, 32, 1)
data = pickle.load(open('data.pickle', 'rb'))
print("Data of format:")
for ch in data:
    print("data_length(" + ch + ") =", len(data[ch]))

#no. of classes
classes = len(data.keys())

Data of format:
data_length(श) = 2000
data_length(ण) = 2000
data_length(त्र) = 2000
data_length(ञ) = 2000
data_length(न) = 2000
data_length(ह) = 2000
data_length(भ) = 2000
data_length(प) = 2000
data_length(फ) = 2000
data_length(ख) = 2000
data_length(ङ) = 2000
data_length(द) = 2000
data_length(स) = 2000
data_length(ग) = 2000
data_length(क) = 2000
data_length(थ) = 2000
data_length(ल) = 2000
data_length(ढ) = 2000
data_length(घ) = 2000
data_length(च) = 2000
data_length(व) = 2000
data_length(ड) = 2000
data_length(म) = 2000
data_length(झ) = 2000
data_length(ज) = 2000
data_length(य) = 2000
data_length(ट) = 2000
data_length(ध) = 2000
data_length(ब) = 2000
data_length(त) = 2000
data_length(क्ष) = 2000
data_length(ठ) = 2000
data_length(ष) = 2000
data_length(छ) = 2000
data_length(ज्ञ) = 2000
data_length(र) = 2000


# Rearranging data in input-output format

In [3]:
file = "class_numerals_{}.pickle".format(classes)
if file in os.listdir(pickle_folder):
    class_numerals = pickle.load(open(os.path.join(pickle_folder, file), 'rb'))
    print("Unpickled")
else:
    class_numerals = {i:list(data.keys())[i] for i in range(classes)}
    pickle.dump(class_numerals, open(os.path.join(pickle_folder, file), 'wb'))
    print("Error. Incompatible. Pickled new file:", file)
def character_to_encoding(char, classes=classes):
    for i in class_numerals:
        if class_numerals[i] == char:
            return np_utils.to_categorical(i, classes)

def encoding_to_character(encoding):
    return class_numerals[encoding.argmax()]

Unpickled


In [4]:
X = []
y = []

for ch in data:
    X += data[ch]
    y += [ch] * len(data[ch])
    
#resizing input image and preprocessin
X = np.array(list(map(lambda x: cv2.resize(x/255, (input_shape[0], input_shape[1])).reshape((input_shape[0], input_shape[1], 1)), X)))
y_ = np.array(list(map(lambda x: character_to_encoding(x, classes), y)))

print(X.shape, y_.shape)

(72000, 32, 32, 1) (72000, 36)


# Train-Test split

In [5]:
train_ratio = 0.8
X_train, X_test, y_train, y_test = train_test_split(X, y_, train_size=0.8, random_state=20, shuffle=True, stratify=y)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((57600, 32, 32, 1), (57600, 36), (14400, 32, 32, 1), (14400, 36))

# CNN

In [6]:
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


print("GPUs:", k.tensorflow_backend._get_available_gpus())

print("input shape:", input_shape)
print("classes:", classes)

def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

GPUs: []
input shape: (32, 32, 1)
classes: 36


loading the model

In [7]:
path_to_model = "pickle files/36/Lenet_ब_य_ञ_त_क्ष_फ_ठ_स_छ_ज्ञ_च_ण_द_र_ट_म_प_क_ङ_भ_न_ल_ह_त्र_ज_ख_झ_थ_घ_श_ष_ग_व_ढ_ड_ध.h5"
model = keras.models.load_model(os.path.join(path_to_model))

In [8]:
temp_test = X_test, y_test
# temp_test = X_train, y_train
a = model.predict_classes(temp_test[0])

In [9]:
g = lambda n: [0]*n + [1] + [0]*(35-n)
X_test, y_test = temp_test
y_pred = np.array(list(map(g, a)))
y_pred = np.array(list(map(encoding_to_character, y_pred)))
y_test = np.array(list(map(encoding_to_character, y_test)))
print(y_test.shape, y_pred.shape)

falsify = ~(y_test == y_pred)
y_pred = y_pred[falsify]
y_test = y_test[falsify]
X_test = X_test[falsify]
X_test = X_test.reshape(X_test.shape[:-1])
print(y_test.shape, y_pred.shape, X_test.shape)

(14400,) (14400,)
(683,) (683,) (683, 32, 32)


In [29]:
per_character_accuracy = {i: 0 for i in data.keys()}

In [30]:
for character in np.unique(y_test):
    per_character_accuracy[character] = 1-sum(y_test == character)/400
#     wrongly_predicted = pd.Series(y_pred[y_test == character]).value_counts().argmax()
#     display_dataframe(pd.DataFrame(pd.Series(y_pred[y_test == character]).value_counts()).transpose())
#     display_all_in_a_list(X_test[(y_test==character) & (y_pred==wrongly_predicted)])
#     print("\n\n\n\n\n\n")

In [31]:
character_frequency = {'क': 4468,
 'क्ष': 125,
 'ख': 341,
 'ग': 679,
 'घ': 122,
 'ङ': 0,
 'च': 661,
 'छ': 194,
 'ज': 848,
 'ज्ञ': 25,
 'झ': 117,
 'ञ': 3,
 'ट': 294,
 'ठ': 128,
 'ड': 270,
 'ढ': 92,
 'ण': 316,
 'त': 3120,
 'त्र': 143,
 'थ': 543,
 'द': 1260,
 'ध': 490,
 'न': 3236,
 'प': 1858,
 'फ': 84,
 'ब': 716,
 'भ': 805,
 'म': 1921,
 'य': 1553,
 'र': 4116,
 'ल': 1128,
 'व': 1850,
 'श': 589,
 'ष': 361,
 'स': 2645,
 'ह': 3178}

In [32]:
accuracy = 0
total_characters = 0
for ch in per_character_accuracy:
    accuracy += per_character_accuracy[ch] * character_frequency[ch]
    total_characters += character_frequency[ch]
    
print(total_characters, accuracy/total_characters)

38279 0.9622790564016824
